In [1]:
import pandas as pd
import numpy as np
from sklearn.utils import class_weight
import os

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras import metrics
from keras import backend as K
from keras.models import Model

#https://machinelearningmastery.com/how-to-use-transfer-learning-when-developing-convolutional-neural-network-models/
from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
from keras.applications.densenet import DenseNet169

#from wcs.google import google_drive_share
import urllib.request
from urllib.parse import urlparse

import warnings
warnings.simplefilter(action='ignore')
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [2]:
# Config
DIR = './'
DATA_DIR_POSTER = DIR + '../data/raw/posters_v3/'
DATA_DIR_INTERIM = DIR + "../data/interim/"
DATA_DIR_RAW = DIR + "../data/raw/"
MODEL_DIR = DIR + "../models/"
BASE_DIR = DIR
IMAGES_DIR = DATA_DIR_POSTER

In [3]:
# Def metrics
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [4]:
# Preprocessing
df = pd.read_parquet(DATA_DIR_INTERIM + "df_train_balanced_v3.gzip")

# Setuup data generators
datagen = ImageDataGenerator(rescale=1 / 255., validation_split=0.1)
BATCH_SIZE = 64
train_generator = datagen.flow_from_dataframe(
    dataframe=df,
    directory=IMAGES_DIR,
    x_col="filename",
    y_col="genre_id",
    batch_size=BATCH_SIZE,
    seed=42,
    shuffle=True,
    class_mode="categorical",
    target_size=(299, 299),
    subset='training',
    validate_filenames=True
)

valid_generator = datagen.flow_from_dataframe(
    dataframe=df,
    directory=IMAGES_DIR,
    x_col="filename",
    y_col="genre_id",
    batch_size=BATCH_SIZE,
    seed=42,
    shuffle=True,
    class_mode="categorical",
    target_size=(299, 299),
    subset='validation',
    validate_filenames=True
)

# Setup class weights
class_weights = class_weight.compute_class_weight('balanced',
                                                  np.array(list(train_generator.class_indices.keys()),dtype="int"), 
                                                  np.array(df.genre_id.explode(),dtype="int"))
                                                 #np.unique(y_train),
                                                 #y_train)
class_weights_genre_id = dict(zip(list(train_generator.class_indices), class_weights))
display(class_weights_genre_id)
class_weights = dict(zip(list(range(len(class_weights))), class_weights))

Found 13239 validated image filenames belonging to 19 classes.
Found 1470 validated image filenames belonging to 19 classes.


{12: 0.9582155284906481,
 14: 1.2180599232816118,
 16: 1.060666245373296,
 18: 0.3579902090231368,
 27: 1.0810636731689363,
 28: 0.7519680839715828,
 35: 0.5943945091822797,
 36: 1.5446338577501204,
 37: 2.0033534159372515,
 53: 0.8413175796634443,
 80: 1.1215872207726088,
 99: 1.2500709320470988,
 878: 1.2785012151329391,
 9648: 1.340393976270155,
 10402: 1.5106720384021943,
 10749: 0.9572266579762099,
 10751: 1.1188813408672464,
 10752: 1.6926142912024587,
 10770: 1.396916613823716}

In [5]:
# Build model
tf.debugging.set_log_device_placement(False)
strategy = tf.distribute.MirroredStrategy()
with strategy.scope():
    model = DenseNet169(include_top=False,  
                input_shape=(299,299, 3), 
                weights=f"{MODEL_DIR}densenet169_weights_tf_dim_ordering_tf_kernels_notop.h5")

    x= layers.Flatten()(model.layers[-1].output)
    #x= layers.Dense(128, activation='relu')(x)
    x= layers.Dense(1024, activation='relu')(x)
    x= layers.Dense(128, activation='relu')(x)
    #x= layers.Dropout(0.5)(x)
    classifications = layers.Dense(len(train_generator.class_indices), activation='sigmoid')(x)
    model = Model(inputs=model.inputs, outputs=classifications)

    # mark loaded layers as not trainable
    # except last layer
    leng = len(model.layers)
    print(leng)
    for i,layer in enumerate(model.layers):
        if leng-i == 500:
          print("stopping at",i)
          break
        layer.trainable = False

    print(model.summary())
    
    # Compile model
    model.compile(optimizer='adam', loss="binary_crossentropy", 
          metrics=[metrics.categorical_accuracy,
                   tf.keras.metrics.AUC(),
                    tf.keras.metrics.Precision(), 
                    tf.keras.metrics.Recall(),
                    f1_m,
                  ])

    # Train model
    history = model.fit(
        train_generator,
        #validation_data=valid_generator,
        epochs=20,# 5,
        #batch_size=BATCH_SIZE,
        #steps_per_epoch=1430,
        class_weight = class_weights
    )       

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensor

InvalidArgumentError: No OpKernel was registered to support Op 'NcclAllReduce' used by {{node NcclAllReduce}} with these attrs: [reduction="sum", shared_name="c1", T=DT_FLOAT, num_devices=2]
Registered devices: [CPU, GPU, XLA_CPU, XLA_GPU]
Registered kernels:
  <no registered kernels>

	 [[NcclAllReduce]] [Op:__inference_train_function_151709]

In [ ]:
tf.debugging.set_log_device_placement(True)

strategy = tf.distribute.MirroredStrategy()
with strategy.scope():
  inputs = tf.keras.layers.Input(shape=(1,))
  predictions = tf.keras.layers.Dense(1)(inputs)
  model = tf.keras.models.Model(inputs=inputs, outputs=predictions)
  model.compile(loss='mse',
                optimizer=tf.keras.optimizers.SGD(learning_rate=0.2))